In [1]:
# Make sure the dataset is connected, Kaggle only

!ls -la ../input/tensorflow-great-barrier-reef
!ls -ls ../input/pretrained-master-nongraph

In [2]:
# Pull in python files, Kaggle only

!rsync -v ../input/pretrained-master-nongraph/*.py .
!ls -la .

In [3]:
# Dummy class to quickly debug the Kaggle interface for our code

class DummyModel():
    def __init__(self):
        self.outstring = "0.5 0 0 0 0"

    def predict(self, minibatch):
        print(minibatch.shape)
        return self.outstring

In [4]:
import os, faster_rcnn
import tensorflow as tf

backbone_weights='../input/pretrained-master-nongraph/trained_backbone.ckpt'
rpn_weights='../input/pretrained-master-nongraph/trained_rpn.ckpt'
classifier_weights='../input/pretrained-master-nongraph/trained_classifier.ckpt'

frcnn = faster_rcnn.FasterRCNNWrapper(
    input_shape=(720, 1280, 3),
    datapath='../input/tensorflow-great-barrier-reef',
    backbone_type='ResNet50',
    backbone_weights=backbone_weights if os.path.exists(backbone_weights) else 'finetune',
    rpn_weights=rpn_weights if os.path.exists(rpn_weights) else None,
    classifier_weights=classifier_weights if os.path.exists(classifier_weights) else None
)

# Either instantiate the dummy or real model
#frcnn = DummyModel()

In [5]:
# Actual submission call
import greatbarrierreef

env = greatbarrierreef.make_env()
iter_test = env.iter_test()

for (pixel_array, prediction_df) in iter_test:
    prediction_df['annotations'] = frcnn.predict(pixel_array[tf.newaxis,:,:,:])
    env.predict(prediction_df)

In [6]:
!cat submission.csv